In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import xgboost as xgb

pd.set_option('display.float_format',lambda x: '%.5f'%x)

print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close() 

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfPred = dfPred.rename(columns=col_name)
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

#dfAll = dfAll.replace({'':np.nan,0:np.nan})
dfAll = dfAll.reset_index()

Load data...
Done.


In [2]:
dfAll = pd.read_csv('../../Data/All_non_missing_v2.csv')

In [ ]:
dfAll[pro_feat] = dfAll[pro_feat]-dfAll[pro_feat].mean()

In [ ]:
#dfAll = dfAll.replace({-99999999:-2.2222222})
dfAll[['750X1450','750X1451']]

In [ ]:
#dfAll = dfAll.replace({np.nan:-99999999})
pro_feat = [var for var in raw_predictors if not 'TOOL' in var ]
corr = dfAll[pro_feat].corr()
for i in range(len(corr)):
    corr.iloc[i,i:] =np.nan
print((corr==1.0).sum(axis=1))

In [ ]:
corr

In [ ]:
#处理0这个比较异常的值,在trian set中有40条数据，在某些column上为0 ，需要处理，test_a中也有一条
feature = ['ID']
for var in raw_predictors:
    if 'TOOL' in var:
        feature.append(var)
pick_list = (((corr==1.0).sum(axis=1))==0).values
for i in range(len(pick_list)):
    if pick_list[i]==True:
        feature.append(corr.index[i])
print(len(feature))

In [ ]:
#number of NAN
dfAll['Total_NAN_Num']=dfAll[raw_predictors].T.isnull().sum().tolist()
for key,pro in procedure.items():
    dfAll[key+'_NAN_Num']=dfAll[pro].T.isnull().sum().tolist()

In [ ]:
raw_predictors = dfAll.columns.tolist()[1:-13]
raw_predictors[-1]

In [ ]:
#Count 计算特定值在变量中重复出现的次数'''
dfAll = dfAll.replace({np.nan:-99999999})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    dfAll['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dfAll.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dfAll['cnt_'+var].ix[group.index] = count
dfAll[added].to_csv('../../Cache/Feat_cnt_col_nonmissing.csv',index=False)

added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
dfAll[added].to_csv('../../Cache/Feat_cnt_tool_nonmissing.csv',index=False)

In [ ]:
#percentile 计算某一个值在全集中的位置（百分制）
dfAll = dfAll.replace({-99999999:np.nan})
added = ['ID']
for var in raw_predictors:
    if not var in dfAll.columns:
        continue
    if 'TOOL' not in var:
        dfAll['pcent_'+var] = dfAll[var].rank(method='max')/float(len(dfAll))
        added.append('pcent_'+var)
dfAll[added].to_csv('../../Cache/Feat_pcent_col_nonmissing.csv',index=False)
t = 0
added = ['ID']
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
            
dfAll[added].to_csv('../../Cache/Feat_pcent_tool_nonmissing.csv',index=False) 

In [ ]:
len(raw_predictors)

In [ ]:
minus_var = ['ID']
for i in range(len(raw_predictors)-1):
    for j in range(i+1,len(raw_predictors)):
        if (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].max())*(dfAll[raw_predictors[i]].min()-dfAll[raw_predictors[j]].min())<0:
            if float(min(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min()))/max(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[i]].min(),dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[j]].min())>=0.6:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)
        elif (dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min())*(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())>0:
            if float(min(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min())))/max(abs(dfAll[raw_predictors[i]].max()-dfAll[raw_predictors[j]].min()),abs(dfAll[raw_predictors[j]].max()-dfAll[raw_predictors[i]].min()))>=0.6:
                tmp_var = raw_predictors[i]+'-'+raw_predictors[j]
                dfAll[tmp_var] = dfAll[raw_predictors[i]]-dfAll[raw_predictors[j]]
                minus_var.append(tmp_var)
                print(tmp_var)

In [ ]:
dffAll_minus = dfAll[minus_var]
dffAll_minus.to_csv('../../../All_minus.csv',index=False)

In [3]:
dffAll_minus = pd.read_csv('../../../All_minus.csv')

In [4]:
minus_var = dffAll_minus.columns.tolist()

In [6]:
added = ['ID']
t=0
for var in minus_var[1:]:
    if dffAll_minus[var].nunique()>10:
        continue
    t+=1
    print(t)
    dffAll_minus['cnt_'+var] = 0
    added.append('cnt_'+var)
    groups = dffAll_minus.groupby([var])
    for name,group in groups:
        count = group[var].count()
        dffAll_minus['cnt_'+var].ix[group.index] = count
for var in minus_var[1:]:
    if 'TOOL' not in var:
        dffAll_minus['pcent_'+var] = dffAll_minus[var].rank(method='max')/float(len(dffAll_minus))
        added.append('pcent_'+var)

1


C:\Users\Leo Mao\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  del sys.path[0]
C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062


3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701


KeyError: '210X1-220X392'

In [7]:
for var in minus_var[1:]:
    if 'TOOL' not in var:
        dffAll_minus['pcent_'+var] = dffAll_minus[var].rank(method='max')/float(len(dffAll_minus))
        added.append('pcent_'+var)

In [10]:
dffAll_minus[added].to_csv('../../../All_minus_single_col.csv',index=False)

In [5]:
tool = ['ID']
for var in dfAll:
    if 'TOOL' in var:
        tool.append(var)
dffAll_minus = dffAll_minus.merge(dfAll[tool],'inner','ID')
dffAll_minus.shape

(721, 41860)

In [6]:
minus_tool_dict ={}

for var1 in tool[1:]:
    var1_num = var1[-3:]
    minus_tool_dict[var1_num] ={}
    tmp = [i for i in dffAll_minus.columns if i[0:3]==var1_num]
    for var2 in tool[1:]:
        var2_num = var2[-3:]
        minus_tool_dict[var1_num][var2_num] =[i for i in tmp if i[i.find('-')+1:i.find('-')+4]==var2_num]
minus_tool_dict
        

{'210': {'210': ['210X2-210X37',
   '210X2-210X80',
   '210X2-210X226',
   '210X2-210X229',
   '210X3-210X89',
   '210X3-210X90',
   '210X3-210X158',
   '210X3-210X159',
   '210X3-210X176',
   '210X3-210X177',
   '210X4-210X12',
   '210X4-210X107',
   '210X4-210X142',
   '210X4-210X143',
   '210X5-210X39',
   '210X6-210X52',
   '210X6-210X107',
   '210X6-210X142',
   '210X7-210X8',
   '210X12-210X105',
   '210X12-210X143',
   '210X13-210X14',
   '210X13-210X15',
   '210X13-210X16',
   '210X13-210X32',
   '210X13-210X34',
   '210X14-210X15',
   '210X14-210X16',
   '210X14-210X32',
   '210X14-210X33',
   '210X14-210X34',
   '210X14-210X35',
   '210X15-210X16',
   '210X15-210X32',
   '210X15-210X34',
   '210X16-210X32',
   '210X16-210X34',
   '210X19-210X210',
   '210X20-210X23',
   '210X25-210X26',
   '210X25-210X128',
   '210X25-210X129',
   '210X25-210X200',
   '210X25-210X201',
   '210X25-210X202',
   '210X25-210X203',
   '210X25-210X211',
   '210X25-210X212',
   '210X26-210X128',
   

In [ ]:
added = ['ID']
for key1,value1 in minus_tool_dict.items():
    for key2,value2 in value1.items():
        if key1==key2:
            groups = dffAll_minus.groupby(['TOOL_'+key1])
        else:
            groups = dffAll_minus.groupby(['TOOL_'+key1,'TOOL_'+key2])
        print(key1,key2)
        for var in value2:
            dffAll_minus['cnt_tool_'+var] = 0
            dffAll_minus['pcent_tool_'+var] = 0
            added.append('cnt_tool_'+var)
            added.append('pcent_tool_'+var)
            for name,group in groups:
                dffAll_minus['pcent_tool_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))
                grps = group.groupby([var])
                for name2,grp in grps:
                    dffAll_minus['cnt_tool_'+var].ix[grp.index] = float(len(grp))/float(len(group))

    
'''for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]])
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['cnt_'+tool+'_'+var] = 0
        added.append('cnt_'+tool+'_'+var)
        for name,group in groups:
            grps = group.groupby([var])
            for name2,grp in grps:
                dfAll['cnt_'+tool+'_'+var].ix[grp.index] = float(len(grp))/float(len(group))
                
                
                
                
for tool,pro in procedure.items():
    if tool[-3:] in ['261','400','420']:
        continue
    groups = dfAll.groupby(['TOOL_'+tool[-3:]]) 
    for var in pro:
        if not var in dfAll.columns:
            continue
        dfAll['pcent_'+tool+'_'+var] = 0
        added.append('pcent_'+tool+'_'+var)
        for name,group in groups:
            dfAll['pcent_'+tool+'_'+var].ix[group.index] = group[var].rank(method='max')/float(len(group))'''

210 210


C:\Users\Leo Mao\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  from ipykernel import kernelapp as app
C:\Users\Leo Mao\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\Leo Mao\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


210 220
210 300
210 310
210 311
210 312
210 330
210 340
210 344
210 360
210 440
210 520
210 750
220 210
220 220
220 300
220 310
220 311
220 312
220 330
220 340
220 344
220 360
220 440
220 520
220 750
300 210
300 220
300 300
300 310
300 311
300 312
300 330
300 340
300 344
300 360
300 440
300 520
300 750
310 210
310 220
310 300
310 310
310 311
310 312
310 330
310 340
310 344
310 360
310 440
310 520
310 750
311 210
311 220
311 300
311 310
311 311
311 312
311 330
311 340
311 344
311 360
311 440
311 520
311 750
312 210
312 220
312 300
312 310
312 311
312 312
312 330
312 340
312 344
312 360
312 440
312 520
312 750
330 210
330 220
330 300
330 310
330 311
330 312
330 330


In [ ]:
#Pivot 创建多组category variable 的两两交叉表,感觉只能用来计算分类型变量,好像现在直接可以用pandas解决
def get_pivottable(X_train, X_test, use='all',Feat_list):
    """
    Returns a list of dictionaries, one per feature in the
    basic data, containing cross-tabulated counts
    for each column and each value of the feature.
    """
    dictionaries = []
    if use == 'all':
        X = np.vstack((X_train, X_test))
        filename = "pivottable"
    elif use == 'train':
        X = X_train
        filename = "pivottable_train"
    else:
        X = X_test
        filename = "pivottable_test"
    for i in range(len(Feat_list)):
        dictionaries.append({'total': 0})
    try:
        with open("cache/%s.pkl" % filename, 'rb') as f:
            logger.debug("loading cross-tabulated data from cache")
            dictionaries = pickle.load(f)
    except IOError:
        logger.debug("no cache found, cross-tabulating data")
        for i, row in enumerate(X):
            for j in range(len(Feat_list)):
                dictionaries[j]['total'] += 1
                if row[j] not in dictionaries[j]:
                    dictionaries[j][row[j]] = {'total': 1}
                    for k, key in enumerate(Feat_list):
                        dictionaries[j][row[j]][key] = {row[k]: 1}
                else:
                    dictionaries[j][row[j]]['total'] += 1
                    for k, key in enumerate(Feat_list):
                        if row[k] not in dictionaries[j][row[j]][key]:
                            dictionaries[j][row[j]][key][row[k]] = 1
                        else:
                            dictionaries[j][row[j]][key][row[k]] += 1
        with open("cache/%s.pkl" % filename, 'wb') as f:
            pickle.dump(dictionaries, f, pickle.HIGHEST_PROTOCOL)

    return dictionaries

def creat_cate_corr(X_train, X_test, Feat_list=[var if 'TOOL' in var for var in raw_predictor]):
    features_train = []
    features_test = []
    dictionaries = get_pivottable(X_train[Feat_list], X_test[Feat_list],Feat_list=Feat_list)
    #dictionaries_train = get_pivottable(X_train[Feat_list], X_test[Feat_list], use='train',Feat_list=Feat_list)
    #dictionaries_test = get_pivottable(X_test[Feat_list], X_test[Feat_list], use='test',Feat_list=Feat_list)

In [ ]:
dfAll = pd.read_csv('../../Data/All_Data_dedup.csv')
feat_needed = dfAll.columns.tolist()[14:]
'''gap_list = []
for i in range(len(feat_needed)-1):
    for j in range(i+1,len(feat_needed)):
        if dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)<=1 and dfAll[feat_needed[i]].quantile(0.5) - dfAll[feat_needed[j]].quantile(0.5)>=-1:
            tmp_var = feat_needed[i]+'-'+feat_needed[j]
            dfAll[tmp_var] = dfAll[feat_needed[i]]- dfAll[feat_needed[j]]
            gap_list.append(tmp_var)
            print(tmp_var)'''

In [ ]:
feat_needed2 =[]
for var in feat_needed:
    if dfAll[var].nunique()>15:
        feat_needed2.append(var)
gap_list = []
for i in range(len(feat_needed2)-1):
    for j in range(i+1,len(feat_needed2)):
        if dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)<=2 and dfAll[feat_needed2[i]].quantile(0.5) - dfAll[feat_needed2[j]].quantile(0.5)>=-2:
            tmp_var = feat_needed2[i]+'-'+feat_needed2[j]
            dfAll[tmp_var] = dfAll[feat_needed2[i]]- dfAll[feat_needed2[j]]
            gap_list.append(tmp_var)
            print(tmp_var)

In [ ]:
dfAll = dfAll.set_index(dfAll['ID'])
dfTrain_tmp = dfAll.loc[dfTrain['ID']]
dfPred_tmp = dfAll.loc[dfPred['ID']]

In [ ]:
dfPred = dfPred_tmp.copy(deep=True)
dfTrain = dfTrain_tmp.copy(deep=True)

In [ ]:
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')

In [ ]:
dfAll2 = pd.read_csv('../../Data/All_Data_dedup.csv',nrows=2)
raw_predictors = dfAll2.columns[1:]

In [ ]:
dfTrain.columns

In [ ]:
f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()
predictors = dfTrain.columns.tolist()[1:]+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        try:
             predictors.remove(var)
        except:
            continue
    for var in var_change[key]['category']:
        try:
            predictors.remove(var)
        except:
            continue
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [ ]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':3, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':1,'subsample':0.8,
                         'colsample_bytree':0.4}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    #print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [ ]:
dfTrain2 = pd.read_csv(config.original_train_data_path,usecols=['ID','Y'])
dfTrain = dfTrain.merge(dfTrain2,'inner','ID')
dfTrain.shape

In [ ]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)
imp.to_csv('../../Profiling/imp_test_v3_%s.csv'%today,index=False)
test_result('../../Profiling/test_score_%s.csv'%today,index=False)